In [1]:
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torchvision
from torchvision import transforms

In [12]:
img_Gray = Image.open('C:/Paper_Project/Hangul_Characters_Image128_Grayscale/가/62570_가.png')
img_Gray_array = np.array(img_Gray)

print(img_Gray_array.shape)

(128, 128)


In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline


img = cv2.imread('C:/Paper_Project/1.png')



In [3]:
img.shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [7]:
import torch
import torch.nn as nn

# context와 x의 예시 shape
context_shape = (16, 68, 256)
x_shape = (16, 256, 16, 16)


# 임의의 값으로 채워진 context와 x 텐서 생성
context = torch.rand(context_shape)
x = torch.rand(x_shape)

# 예시로 nn.Linear를 사용하여 context를 변환
# nn.Linear를 사용하여 context 텐서 변환
linear_layer = nn.Linear(context_shape[-1], x_shape[1] * x_shape[2] * x_shape[3])
context_transformed = linear_layer(context)

# 변환된 context를 원하는 크기로 reshape
desired_shape = (x_shape[0], x_shape[1], x_shape[2], x_shape[3])
context_reshaped = context_transformed.view(desired_shape)

torch.Size([16, 68, 1, 256])


RuntimeError: Sizes of tensors must match except in dimension 2. Expected size 68 but got size 256 for tensor number 1 in the list.